In [572]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f'Using device: {device} - {torch.cuda.get_device_name(0)}')
else:
    raise Exception("GPU is not available. Please run this notebook on a system with a GPU.")

Using device: cuda:0 - NVIDIA GeForce RTX 3080


In [573]:
class FingerprintDataset(Dataset):
    def __init__(self, real_dir, altered_dir, transform=None, limit=None):
        self.real_dir = real_dir
        self.altered_dir = altered_dir
        self.transform = transform
        self.limit = limit

        self.real_images = os.listdir(real_dir)[:limit] if limit else os.listdir(real_dir)
        self.altered_images = [self.find_altered(real_img) for real_img in self.real_images]
        self.real_images, self.altered_images = zip(*[(real, altered) for real, altered in zip(self.real_images, self.altered_images) if altered is not None])

    def find_altered(self, real_img):
        for file in os.listdir(self.altered_dir):
            if real_img.split('.')[0] in file:
                return file
        return None

    def __len__(self):
        return len(self.real_images)

    #TODO LOGIC TO MATCH REAL FINGERPRINT WITH ALTERED FINGERPRINT

    def __getitem__(self, idx):
        if idx >= len(self.real_images):
            raise IndexError('Index out of range')

        real_img = self.real_images[idx]
        real_image = Image.open(os.path.join(self.real_dir, real_img))

        altered_img = self.altered_images[idx]
        altered_image = Image.open(os.path.join(self.altered_dir, altered_img))

        if self.transform:
            real_image = self.transform(real_image)
            altered_image = self.transform(altered_image)

        return real_image, altered_image, 1

In [574]:
transform = transforms.Compose([
    transforms.Grayscale(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values to [-1, 1]
])

In [575]:
real_dir = 'dataset/Real'
altered_dir = 'dataset/Altered/Altered-Easy'

dataset = FingerprintDataset(real_dir, altered_dir, transform=transform)

#TODO SET NUMBER OF THREADS ACCORDINGLY

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=6)

In [576]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        self.feature_extractor = models.resnet18(pretrained=True)
        
        self.feature_extractor.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        self.feature_extractor.fc = nn.Identity()
        
        # Define the final layer that computes the absolute difference between the two outputs
        self.final_layer = nn.Linear(512, 1)

    def forward(self, input1, input2):
        # Ensure the input has at least 4 dimensions
        if len(input1.shape) < 4:
            input1 = input1.unsqueeze(1)
        if len(input2.shape) < 4:
            input2 = input2.unsqueeze(1)

        # Ensure the input has the correct dimensions
        input1 = input1.view(input1.shape[0], -1, input1.shape[2], input1.shape[3])
        input2 = input2.view(input2.shape[0], -1, input2.shape[2], input2.shape[3])

        # Pass both inputs through the feature extractor
        output1 = self.feature_extractor(input1)
        output2 = self.feature_extractor(input2)
        
        # Compute the absolute difference between the two outputs
        diff = torch.abs(output1 - output2)
        
        # Pass the difference through the final layer to get the output
        output = self.final_layer(diff)
        
        return output

In [577]:
model = SiameseNetwork().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [578]:
num_epochs = 10
running_loss = 0.0

for epoch in range(num_epochs):
    for i, (inputs1, inputs2, labels) in enumerate(dataloader):
        inputs1 = inputs1.to(device)
        inputs2 = inputs2.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs1, inputs2)
        outputs = outputs.squeeze()  # Remove the extra dimension

        labels = labels.float().squeeze()
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / (i+1)))

    running_loss = 0.0

    

print('Finished Training')

[1,     1] loss: 0.496
[1,     2] loss: 0.445
[1,     3] loss: 0.404
[1,     4] loss: 0.357
[1,     5] loss: 0.308
[1,     6] loss: 0.268
[1,     7] loss: 0.237
[1,     8] loss: 0.213
[1,     9] loss: 0.194
[1,    10] loss: 0.177
[1,    11] loss: 0.164
[1,    12] loss: 0.151
[1,    13] loss: 0.140
[1,    14] loss: 0.131
[1,    15] loss: 0.122
[1,    16] loss: 0.115
[1,    17] loss: 0.109
[1,    18] loss: 0.103
[1,    19] loss: 0.098
[1,    20] loss: 0.093
[1,    21] loss: 0.089
[1,    22] loss: 0.085
[1,    23] loss: 0.081
[1,    24] loss: 0.079
[1,    25] loss: 0.076
[1,    26] loss: 0.073
[1,    27] loss: 0.070
[1,    28] loss: 0.068
[1,    29] loss: 0.066
[1,    30] loss: 0.064
[1,    31] loss: 0.061
[1,    32] loss: 0.060
[1,    33] loss: 0.058
[1,    34] loss: 0.056
[1,    35] loss: 0.055
[1,    36] loss: 0.053
[1,    37] loss: 0.052
[1,    38] loss: 0.050
[1,    39] loss: 0.049
[1,    40] loss: 0.048
[1,    41] loss: 0.047
[1,    42] loss: 0.046
[1,    43] loss: 0.044
[1,    44] 